<a href="https://colab.research.google.com/github/rafikg/BusinessCardReader/blob/master/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall


In [9]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [10]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [ ]:
class F1Score(tf.keras.metrics.Metric):

    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.f1 = self.add_weight(name='f1', initializer='zeros')
        self.precision_fn = Precision(thresholds=0.5)
        self.recall_fn = Recall(thresholds=0.5)

    def update_state(self, y_true, y_pred, sample_weight=None):
        p = self.precision_fn(y_true, y_pred)
        r = self.recall_fn(y_true, y_pred)
        self.f1 = 2 * ((p * r) / (p + r + 1e-6))

    def result(self):
        return self.f1

    def reset_states(self):
        self.f1 = 0.0

In [11]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy", Precision(),
                           Recall(), F1Score()])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)



Epoch 1/15
422/422 [==============================] - ETA: 0s - loss: 0.3743 - accuracy: 0.8850 - precision: 0.9438 - recall: 0.8349 - f1_score: 0.8860

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/metrics.py:257: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))


422/422 [==============================] - 36s 9ms/step - loss: 0.3743 - accuracy: 0.8850 - precision: 0.9438 - recall: 0.8349 - f1_score: 0.8860 - val_loss: 0.0890 - val_accuracy: 0.9757 - val_precision: 0.9813 - val_recall: 0.9700 - val_f1_score: 0.8954
Epoch 2/15
422/422 [==============================] - 3s 7ms/step - loss: 0.1155 - accuracy: 0.9651 - precision: 0.9712 - recall: 0.9599 - f1_score: 0.9294 - val_loss: 0.0633 - val_accuracy: 0.9822 - val_precision: 0.9859 - val_recall: 0.9800 - val_f1_score: 0.9322
Epoch 3/15
422/422 [==============================] - 3s 7ms/step - loss: 0.0860 - accuracy: 0.9734 - precision: 0.9775 - recall: 0.9699 - f1_score: 0.9453 - val_loss: 0.0518 - val_accuracy: 0.9872 - val_precision: 0.9888 - val_recall: 0.9852 - val_f1_score: 0.9467
Epoch 4/15
422/422 [==============================] - 3s 7ms/step - loss: 0.0723 - accuracy: 0.9776 - precision: 0.9803 - recall: 0.9751 - f1_score: 0.9540 - val_loss: 0.0410 - val_accuracy: 0.9897 - val_precisio

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
